In [8]:
import torch
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import os
import time

In [3]:
import os
import random
import shutil

# Define the source directories
source_dir_chair = "./dataset/chair/"
source_dir_monitor = "./dataset/monitor/"

# Define the target folders
target_dir_chair_train = "./dataset/chair/train/"
target_dir_chair_test = "./dataset/chair/test/"
target_dir_monitor_train = "./dataset/monitor/train/"
target_dir_monitor_test = "./dataset/monitor/test/"

# Check if the target folders exist, if not create them
if not os.path.exists(target_dir_chair_train):
    os.makedirs(target_dir_chair_train)
if not os.path.exists(target_dir_chair_test):
    os.makedirs(target_dir_chair_test)
if not os.path.exists(target_dir_monitor_train):
    os.makedirs(target_dir_monitor_train)
if not os.path.exists(target_dir_monitor_test):
    os.makedirs(target_dir_monitor_test)

# Get the list of files in the source folders
files_chair = [f for f in os.listdir(source_dir_chair) if os.path.isfile(os.path.join(source_dir_chair, f))]
files_monitor = [f for f in os.listdir(source_dir_monitor) if os.path.isfile(os.path.join(source_dir_monitor, f))]

random.shuffle(files_chair)
random.shuffle(files_monitor)

# Split the files into train and test
split_ratio = 0.8
split_index_chair = int(len(files_chair) * split_ratio)
split_index_monitor = int(len(files_monitor) * split_ratio)

train_files_chair = files_chair[:split_index_chair]
test_files_chair = files_chair[split_index_chair:]

train_files_monitor = files_monitor[:split_index_monitor]
test_files_monitor = files_monitor[split_index_monitor:]

# Copy the files to the target folders
for file in train_files_chair:
    shutil.copy(os.path.join(source_dir_chair, file), os.path.join(target_dir_chair_train, file))
for file in test_files_chair:
    shutil.copy(os.path.join(source_dir_chair, file), os.path.join(target_dir_chair_test, file))
for file in train_files_monitor:
    shutil.copy(os.path.join(source_dir_monitor, file), os.path.join(target_dir_monitor_train, file))
for file in test_files_monitor:
    shutil.copy(os.path.join(source_dir_monitor, file), os.path.join(target_dir_monitor_test, file))

In [9]:
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid

In [10]:
# --- Image Settings ---
img_width, img_height = 224, 224  # Adjust if needed
batch_size = 32
epochs = 10  # Adjust as needed
device = torch.device("cpu")


# --- Data Transformations ---
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.RandomHorizontalFlip(),  # Augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# --- Data Loaders --- (Corrected)
train_dir = "train"  # Path to your main training directory
test_dir = "test"   # Path to your main testing directory

train_dataset = datasets.ImageFolder(train_dir, data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, data_transforms['test'])

dataloaders = {
    'train': torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4),
    'test': torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
}

dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'test']}
class_names = train_dataset.classes

In [4]:
# --- TensorBoard Setup ---
writer = SummaryWriter()

In [5]:

# --- Model Definitions ---

class VGG3Block(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG3Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x64x64 after this layer if input is 128x128


            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x32x32 after this layer



            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 256x16x16 after this layer
        )


        self.classifier = nn.Sequential(
            nn.Linear(256 * (img_width//8) * (img_height//8), 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


class VGG1Block(nn.Module):
    def __init__(self, num_classes=2):  # Make sure num_classes is 2
        super(VGG1Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * (img_width // 2) * (img_height // 2), 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),  # Output layer with num_classes
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the feature maps
        x = self.classifier(x)
        return x



def initialize_model(model_name, num_classes, tune_all_layers=False):

    if model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        num_ftrs = model.classifier[6].in_features  # .classifier if using vgg16/19
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    elif model_name == "vgg19":
        model = models.vgg19(pretrained=True)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    elif model_name == "vgg3block":
        model = VGG3Block(num_classes)
    elif model_name == "vgg1block": # Ensure VGG1Block is defined
        model = VGG1Block(num_classes)  # Instantiate your VGG1Block model
    else:
        raise ValueError("Invalid model name.")


    if not tune_all_layers and 'vgg' in model_name:  # Freeze convolution layers for vgg
        for param in model.features.parameters():
            param.requires_grad = False

    return model.to(device)

In [6]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)


        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)


        time_elapsed = time.time() - since
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


        # Log epoch metrics to Tensorboard:
        writer.add_scalar(f'Loss/{phase}', epoch_loss, epoch)
        writer.add_scalar(f'Accuracy/{phase}', epoch_acc, epoch)


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')



    num_params = sum(p.numel() for p in model.parameters())


    return model, time_elapsed, num_params, best_acc
        
    #return model.to(device)


    '''
            # --- Image Logging (Only for Test Phase) ---
            if phase == 'test':
                # Get a batch of test images (you can modify how many images to log)
                try: # Handle StopIteration if the DataLoader is exhausted.
                    test_iter = iter(dataloaders['test'])
                    images, labels = next(test_iter)
                    images = images.to(device)
                    labels = labels.to(device)


                    # Get predictions
                    with torch.no_grad():
                        outputs = model(images)
                        _, predicted = torch.max(outputs, 1)


                    img_grid = make_grid(images, nrow=8, padding=2, normalize=True, scale_each=True)
                    fig, ax = plt.subplots(figsize=(10, 10))
                    ax.imshow(img_grid.permute(1, 2, 0).cpu().numpy())
                    ax.axis('off')



                    predicted_class_names = [class_names[p] for p in predicted.tolist()]
                    true_class_names = [class_names[l] for l in labels.tolist()]
                    titles = [f"Pred: {p}, True: {t}" for p, t in zip(predicted_class_names, true_class_names)]



                    for i, title in enumerate(titles):
                        plt.text(img_grid.shape[2] * (i % 8) + img_grid.shape[2] // 2,
                                 img_grid.shape[1] * (i // 8) + 10,
                                 title, ha='center', va='bottom', fontsize=6)  # Adjust fontsize as needed


                    writer.add_figure('Test Images with Predictions', fig, global_step=epoch)
                    plt.close(fig)  # Important to free memory!

                except StopIteration:
                    pass # Data loader exhausted. It can happen during last epoch. Handle gracefully.'''


In [7]:
models_to_train = [
    ("vgg1block", False),  # VGG 1-block, no tuning of conv layers
    ("vgg3block", False),  # VGG 3-block, no tuning of conv layers
    ("vgg3block", True),   # VGG 3-block, fine-tune all layers
    ("vgg16", False),      # VGG16, only fine-tune classifier
    ("vgg16", True),       # VGG16, fine-tune all layers
    ("vgg19", False),      # VGG19, only fine-tune classifier
    ("vgg19", True),       # VGG19, fine-tune all layers
]

results = {}
for model_name, tune_all_layers in models_to_train:
    model = initialize_model(model_name, len(class_names), tune_all_layers=tune_all_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Or Adam, etc.

    print(f"Training {model_name}...")
    trained_model, training_time, num_params, best_acc = train_model(model, criterion, optimizer, num_epochs=epochs) # Get metrics

    # --- Evaluation ---
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = trained_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # --- Evaluation --- (Your evaluation code remains the same)


    print(f'Training Time: {training_time:.2f}s')
    print(f'Model Parameters: {num_params}')



    results[model_name + ("_finetuned" if tune_all_layers else "")] = {
        "training_time": training_time,
        "num_params": num_params,
    } #Store other metrics



    torch.save(model.state_dict(), f"{model_name}{'_finetuned' if tune_all_layers else ''}_model.pth")


# --- Print Results Summary ---
print("\n--- Results Summary ---")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  Test Accuracy: {metrics['test_accuracy']:.2f}%")
    print(f"  Training Time: {metrics['training_time']:.2f}s")

    print(f"  Number of Parameters: {metrics['num_params']}")


writer.close()

Training vgg1block...
Epoch 1/10
----------


/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Python(93727) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(93740) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(93741) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(93742) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


test Loss: 1.2307 Acc: 0.6375
Epoch 2/10
----------


Python(95072) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(95073) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(95074) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(95075) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Python(8407) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(8408) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(8409) MallocStackLogging: can't turn off malloc sta

test Loss: 1.2127 Acc: 0.6625
Epoch 3/10
----------


...

Python(78585) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78586) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78587) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78588) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Python(79954) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(79955) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(79956) MallocStackLogging: can't turn off malloc 

test Loss: 0.0017 Acc: 1.0000
Training complete in 29m 38s


Python(80179) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(80180) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(80181) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(80182) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Training Time: 1777.54s
Model Parameters: 139578434

--- Results Summary ---
vgg1block:


KeyError: 'test_accuracy'

In [45]:
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy for {model_name}: {test_accuracy:.2f}%")

Test Accuracy for vgg1block: 68.75%


In [11]:
image_size = 224
num_classes = 2
learning_rate = 0.001
num_epochs = 10

# Define the MLP model
class LargeMLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):  # More flexible initialization
        super(LargeMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)  # Output layer size = num_classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x



# Create the MLP model instance (adjust hidden sizes as needed for desired parameter count)
input_size = image_size * image_size * 3  # Calculate input size based on image dimensions
hidden_size1 = 4096  # Example hidden size, adjust as needed
hidden_size2 = 2048 # Example hidden size, adjust as needed
model = LargeMLP(input_size, hidden_size1, hidden_size2, num_classes).to(device)



# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# TensorBoard writer
#writer = SummaryWriter()


# Training loop
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloaders['train']):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)


        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        print(loss.item())



        # TensorBoard logging (loss per iteration)
        #writer.add_scalar('Training Loss/iterations', loss.item(), epoch * len(dataloaders['train']) + i)


# ... (Add test evaluation and logging as in previous examples)


#writer.close()

Python(64036) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(64037) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(64038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(64039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


0.6614471077919006
183.44561767578125
1276.3084716796875
220.99127197265625
236.43435668945312
56.1220703125
45.41236114501953
48.37205123901367
18.813697814941406
18.798038482666016


Python(65152) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65153) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65154) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65155) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


7.215029716491699
8.938691139221191
2.562096357345581
24.747127532958984
15.999265670776367
10.79984188079834
5.667522430419922
5.385320663452148
4.45949649810791
1.3000036478042603


Python(65865) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65866) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65867) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(65868) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

1.5662784576416016
0.18176357448101044
0.7234828472137451
0.29315754771232605
1.0734360218048096
1.1299101114273071
0.3411537706851959
0.581539511680603
0.9749850034713745
0.09452828019857407


Python(70107) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70108) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70109) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70110) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2.3941235542297363
0.14534099400043488
0.32568085193634033
0.13953596353530884
0.23032155632972717
0.10711938142776489
0.21224258840084076
0.05438525974750519
0.5531640648841858
0.03248869627714157


Python(70745) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70746) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70747) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(70748) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


0.24081912636756897
0.09824293106794357
0.009132783859968185
0.0744587779045105
0.15178710222244263
0.46306511759757996
0.14261268079280853
0.06260806322097778
0.13368014991283417
0.06060353294014931
